In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer
from matplotlib import pyplot
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import hana_ml.dataframe as dataframe
from notebook_hana_connector.notebook_hana_connector import NotebookConnectionContext
conn = NotebookConnectionContext(connectionId = 'hanapoc')

Get data

In [ ]:
sql = (
"select "
#"ORDERS_TOTAL, KITCHENS_TOTAL, NET_VALUE_TOTAL, NET_VALUE_AVG_TOTAL, AVG_DAYS_BTW_ORDERS, DISTRICTS_TOTAL, ONETIME_ORDER, "    
"ORDERS_TOTAL, NET_VALUE_TOTAL, DISTRICTS_TOTAL, "    
"CASE WHEN CUST_CLASS = 'Z0' THEN '0' WHEN CUST_CLASS = 'Z1' then '1' end as CUST_CLASS_INDC, "
#"(PERIODS_TRADED / PERIODS_AVAIL) as ACTIVITY_INDC, "
"DAYS_BETWEEN(FIRST_ORDER, LAST_ORDER) as LIFETIME, "
"ORDERS_TOTAL / (DAYS_BETWEEN(FIRST_ORDER, LAST_ORDER) + 1) as ACTIVITY, "
"(KITCHENS_TOTAL_I - KITCHENS_TOTAL_R) as KITCHENS_TOTAL, "
#"1/DAYS_BETWEEN(LAST_ORDER, TO_DATE('2021-11-10','YYYY-MM-DD')) as RECENCY, "
"CASE WHEN ID_CREDAT = FIRST_ORDER THEN '1' else '0' end as INSTANTSHOP "

"from TBL_BWH_CUSTOMERSUMMARY_ORDATE "
"where NOT_TRADED != '1' and FIRST_ORDER > '2019' and FIRST_ORDER < '2021'" 
"ORDER BY RAND() LIMIT 1000"
)
    
cust_sel = conn.sql(sql)
customers = cust_sel.collect()
customers

Formatting

In [ ]:
customersform = customers
customersform['NET_VALUE_TOTAL'] = customersform['NET_VALUE_TOTAL'].astype(float)
#customersform['NET_VALUE_AVG_TOTAL'] = customersform['NET_VALUE_AVG_TOTAL'].astype(float)
customersform['CUST_CLASS_INDC'] = customersform['CUST_CLASS_INDC'].astype(float)
customersform['ACTIVITY'] = customersform['ACTIVITY'].astype(float)
#customersform['RECENCY'] = customersform['RECENCY'].astype(float)
customersform['INSTANTSHOP'] = customersform['INSTANTSHOP'].astype(float)

In [ ]:
columnspwr = ['ORDERS_TOTAL','NET_VALUE_TOTAL','DISTRICTS_TOTAL','KITCHENS_TOTAL']
columnsquant = ['KITCHENS_TOTAL','INSTANTSHOP','CUST_CLASS_INDC', 'LIFETIME', 'ACTIVITY']
columnsall = columnspwr + columnsquant

Scale the data by quantiles so that distance measures are scaled as prep for clustering

In [ ]:
#scale = MinMaxScaler()
#scalepwr = PowerTransformer(method='yeo-johnson')
scalepwr = QuantileTransformer(n_quantiles=500, random_state=0)
scalequan = QuantileTransformer(n_quantiles=500, random_state=0)

for column in customersform[columnspwr].columns.tolist():
    scaled = 'scaled_' + column
    mod = scalepwr.fit(customersform[column].values.reshape(-1,1))
    customersform[scaled] = mod.transform(customersform[column].values.reshape(-1,1))

for column in customersform[columnsquant].columns.tolist():
    scaled = 'scaled_' + column
    mod = scalequan.fit(customersform[column].values.reshape(-1,1))
    customersform[scaled] = mod.transform(customersform[column].values.reshape(-1,1))

In [ ]:
#cust_sub = cust_scaled[['scaled_ACTIVITY','scaled_NET_VALUE_TOTAL','scaled_KITCHENS_TOTAL','scaled_LIFETIME','scaled_ORDERS_TOTAL']]
cust_scaled = customersform.filter(regex='^scaled', axis=1)
cust_sub = cust_scaled[['scaled_ACTIVITY','scaled_NET_VALUE_TOTAL','scaled_KITCHENS_TOTAL']]

Fit a KMeans clustering with 3 clusters (for vizualisation purposes)

In [ ]:
test = cust_sub.filter(regex='^scaled', axis=1)
km = KMeans(n_clusters=3)
clusters = km.fit_predict(test)
test["cluster"] = clusters

View the 3 clusters from different angles

In [ ]:
fig = plt.figure(figsize = (15,15))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(test[test['cluster'] == 0]['scaled_ACTIVITY'],test[test['cluster'] == 0]['scaled_NET_VALUE_TOTAL'],test[test['cluster'] == 0]['scaled_KITCHENS_TOTAL'], s = 40 , color = 'blue', label = "cluster 0")
ax.scatter(test[test['cluster'] == 1]['scaled_ACTIVITY'],test[test['cluster'] == 1]['scaled_NET_VALUE_TOTAL'],test[test['cluster'] == 1]['scaled_KITCHENS_TOTAL'], s = 40 , color = 'orange', label = "cluster 1")
ax.scatter(test[test['cluster'] == 2]['scaled_ACTIVITY'],test[test['cluster'] == 2]['scaled_NET_VALUE_TOTAL'],test[test['cluster'] == 2]['scaled_KITCHENS_TOTAL'], s = 40 , color = 'green', label = "cluster 2")
ax.set_xlabel('scaled_ACTIVITY')
ax.set_ylabel('scaled_NET_VALUE_TOTAL')
ax.set_zlabel('scaled_KITCHENS_TOTAL')
ax.legend()
#1
#ax.view_init(90, 0)
#2
#ax.view_init(10, 25)
#3
ax.view_init(10, 70)

plt.draw()

Do a proper "elbow"-test for deciding number of clusters

In [ ]:
from sklearn.cluster import KMeans
wcss = []
for k in range(1,11):
    kmeans = KMeans(n_clusters=k, init="k-means++")
    kmeans.fit(cust_scaled)
    wcss.append(kmeans.inertia_)
plt.figure(figsize=(12,6))    
plt.grid()
plt.plot(range(1,11),wcss, linewidth=2, color="red", marker ="8")
plt.xlabel("K Value")
plt.xticks(np.arange(1,11,1))
plt.ylabel("WCSS")
plt.show()

Showing output if model contains more than 3 dimensions...

In [ ]:
def display_parallel_coordinates_centroids(df, num_clusters):
    '''Display a parallel coordinates plot for the centroids in df'''

    # Create the plot
    fig = plt.figure(figsize=(12, 5))
    title = fig.suptitle("Parallel Coordinates plot for the Centroids", fontsize=18)
    fig.subplots_adjust(top=0.9, wspace=0)

    # Draw the chart
    parallel_coordinates(df, 'cluster', color=palette)

    # Stagger the axes
    ax=plt.gca()
    for tick in ax.xaxis.get_major_ticks()[1::2]:
        tick.set_pad(20)   

In [ ]:
def display_parallel_coordinates(df, num_clusters):
    '''Display a parallel coordinates plot for the clusters in df'''

    # Select data points for individual clusters
    cluster_points = []
    for i in range(num_clusters):
        cluster_points.append(df[df.cluster==i])
    
    # Create the plot
    fig = plt.figure(figsize=(12, 15))
    title = fig.suptitle("Parallel Coordinates Plot for the Clusters", fontsize=18)
    fig.subplots_adjust(top=0.95, wspace=0)

    # Display one plot for each cluster, with the lines for the main cluster appearing over the lines for the other clusters
    for i in range(num_clusters):    
        plt.subplot(num_clusters, 1, i+1)
        for j,c in enumerate(cluster_points): 
            if i!= j:
                pc = parallel_coordinates(c, 'cluster', color=[addAlpha(palette[j],0.2)])
        pc = parallel_coordinates(cluster_points[i], 'cluster', color=[addAlpha(palette[i],0.5)])

        # Stagger the axes
        ax=plt.gca()
        for tick in ax.xaxis.get_major_ticks()[1::2]:
            tick.set_pad(20)  

In [ ]:
def addAlpha(colour, alpha):
    '''Add an alpha to the RGB colour'''
    
    return (colour[0],colour[1],colour[2],alpha)

In [ ]:
# Add the cluster number to the original scaled data
X_clustered = pd.DataFrame(cust_scaled, index=cust_scaled.index, columns=cust_scaled.filter(regex='^scaled', axis=1).columns)
X_clustered["cluster"] = clusters

# Display parallel coordinates plots, one for each cluster
display_parallel_coordinates(X_clustered, 3)